![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Length of the movie being rented, in minutes.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [112]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
import matplotlib.pyplot as plt

SEED = 9

# 1. Preprocessing - Getting the # of rental days
df = pd.read_csv('rental_info.csv')
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

# 2. Preprocessing - Adding 2 columns of dummy variables using the special column
df['special_features'] = df['special_features'].astype('str')
df['deleted_scenes'] = np.where(df['special_features'].str.contains('Deleted Scenes'), 1, 0)
df['behind_the_scenes'] = np.where(df['special_features'].str.contains('Behind the Scenes'), 1, 0)

# 3. Assigning the training & testing datasets
subset = df.drop(columns=['rental_date', 'return_date', 'special_features'])
X = subset.drop(columns='rental_length_days')
y = subset['rental_length_days']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

# 4. Performing feature selection by Lasso Regression
lasso = Lasso(alpha=0.01, random_state=SEED, tol=0.0001, max_iter=1000)
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

# Select features with non-zero coefficients
X_train_lasso, X_test_lasso = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

# 5. Choosing models and performing hyperparameter tuning
scaler = StandardScaler()
lr = LinearRegression()
dt = DecisionTreeRegressor(random_state=SEED)
rf = RandomForestRegressor(random_state=SEED)
sgbr = GradientBoostingRegressor(random_state=SEED)
br = BaggingRegressor(random_state=SEED)

models = [lr, dt, rf, sgbr, br, lasso]
mse = []

params_dt = {
    'max_depth': range(1, 50),
    'min_samples_leaf': np.linspace(0.01, 0.5, 50),
    'max_features': np.linspace(0.1, 1.0, 20)
}

params_rf = {
    'n_estimators': range(100, 500, 100),
    'max_depth': range(1, 20),
    'min_samples_leaf': np.linspace(0.01, 0.5, 50),
    'max_features': ['auto', 'sqrt', 'log2']
}

params_sgbr = {
    'n_estimators': range(100, 500, 100),
    'max_depth': range(1, 15),
    'min_samples_leaf': np.linspace(0.01, 0.5, 50),
    'max_features': ['auto', 'sqrt', 'log2'],
    'subsample': np.linspace(0.5, 1.0, 20)
}

params_br = {
    'n_estimators': range(100, 500, 100),
    'max_features': np.linspace(0.1, 1.0, 10),
    'max_samples': np.linspace(0.5, 1.0, 10),
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
    'oob_score': [True, False]
}

params_lasso = {
    'alpha': np.logspace(-4, 0, 50),
    'max_iter': [1000, 5000, 10000],
    'tol': [1e-4, 1e-3, 1e-2]
}

# Mapping models to parameter grids
model_params = {
    lr: None,
    dt: params_dt,
    rf: params_rf,
    sgbr: params_sgbr,
    br: params_br,
    lasso: params_lasso
}

# Evaluate each model
for model, grid in model_params.items():
    if grid:
        cv = RandomizedSearchCV(model, grid, random_state=SEED, cv=10, n_iter=10, n_jobs=-1)
        cv.fit(scaler.fit_transform(X_train_lasso), y_train)
        print(f'Best params for {model.__class__.__name__}: {cv.best_params_}')
        model = cv.best_estimator_
    else:
        model.fit(scaler.fit_transform(X_train_lasso), y_train)

    y_pred = model.predict(scaler.transform(X_test_lasso))
    mse.append(MSE(y_test, y_pred))

print("MSE for each model:", mse)

best_model = model; best_mse = np.min(mse)

Best params for DecisionTreeRegressor: {'min_samples_leaf': 0.01, 'max_features': 0.38421052631578945, 'max_depth': 38}
Best params for RandomForestRegressor: {'n_estimators': 100, 'min_samples_leaf': 0.01, 'max_features': 'sqrt', 'max_depth': 11}
Best params for GradientBoostingRegressor: {'subsample': 0.9210526315789473, 'n_estimators': 300, 'min_samples_leaf': 0.15000000000000002, 'max_features': 'auto', 'max_depth': 6}
Best params for BaggingRegressor: {'oob_score': True, 'n_estimators': 400, 'max_samples': 0.7222222222222222, 'max_features': 1.0, 'bootstrap_features': False, 'bootstrap': True}
Best params for Lasso: {'tol': 0.01, 'max_iter': 1000, 'alpha': 0.0062505519252739694}
MSE for each model: [4.846638768993797, 4.205060840610949, 4.227812332758292, 3.851487812133041, 2.407725382093919, 4.846235169618719]
